In [43]:
from datetime import datetime, timedelta
from pathlib import Path
import os
import sys
import numpy as np
import pandas as pd
import re
import AMBRA_Backups
from bs4 import BeautifulSoup
import AMBRA_Utils
from redcap_funcs import comp_schema_cap_db, get_project_schema

db_name = 'TESTED'
db = AMBRA_Backups.database.Database(db_name)
project_name = 'TESTED Data Collection'
project = AMBRA_Backups.redcap_funcs.get_redcap_project(project_name) 

def q(query, record=None):
    if any(substring in query for substring in ['UPDATE', 'INSERT', 'DELETE']):
        return db.run_insert_query(query, record)
    elif 'SELECT' in query:
        return pd.DataFrame(db.run_select_query(query, record, column_names=True))
    else:
        print('You dont have UPDATE, INSERT, DELETE or SELECT in your query my guy')

In [ ]:
crf_name = 'mr'

schema_questions = pd.DataFrame(db.run_select_query("""SELECT question_text, redcap_variable FROM CRF_Schema_RedCap
                                WHERE crf_name = %s AND question_text IS NOT NULL""", [crf_name], column_names=True))
schema_questions['variable-value'] = schema_questions['redcap_variable']+schema_questions['question_text']

api_questions = pd.DataFrame(project.export_metadata())
api_questions = api_questions[api_questions['form_name'] == crf_name]
field_names = pd.DataFrame(project.export_field_names())
field_names.rename(columns={'original_field_name': 'field_name'}, inplace=True)
# api_questions = pd.merge(field_names, api_questions, on='field_name')
api_questions = pd.merge(api_questions, field_names, on='field_name', how='left')

In [44]:
comp_schema_cap_db(db_name, project_name)

/Users/naderdl/Documents/AMBRA_Backups/AMBRA_Backups/redcap_funcs.py:283: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(row['field_label'], 'html.parser')
/Users/naderdl/Documents/AMBRA_Backups/AMBRA_Backups/redcap_funcs.py:283: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(row['field_label'], 'html.parser')
/Users/naderdl/Documents/AMBRA_Backups/AMBRA_Backups/redcap_funcs.py:283: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(row['field_label'], 'html.parser')



-------------------mr-------------------

The following CRF_Data_RedCap.redcap_variable's are not in CRF_schema_RedCap.redcap_variable's:
['q2018(-10)', 'q2029_mr(-10)', 'q1036_mr(3)']


The following api-metadata question_text's are not in CRF_Schema_RedCap.question_text's:
{redcap_variable : question_text}

{'q2018(-10)': 'White matter hyperintensity (WMH) Periventricular?', 'q2029_mr(-10)': 'White matter hyperintensity (WMH) deep white matter (DWM)?'}




Exception: Please handle the above discrepancies